In [940]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
import math
import statsmodels.api as sm
from statsmodels import regression, stats
import statsmodels
import scipy

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

from scipy.stats import chisquare

In [941]:
DATA_PATH = '/Users/juliusraschke/Documents/Quantitative Finance/Summer Semester 2/Advanced Quant Finance/Data'

### Load in VIX Term Structure

In [942]:
VIX_TS = pd.read_csv(DATA_PATH + '/vixts.csv',skiprows=4)

VIX_TS = VIX_TS.rename({'Date':'DATE','VIX_1^2':'1 month','VIX_2^2':'2 month','VIX_3^2':'3 month','VIX_6^2':'6 month',
               'VIX_9^2':'9 month','VIX_12^2':'12 month'},axis=1)

In [943]:
def DATE_STR(num):
    num_str = str(num)
    date_str = num_str[-2:]+'/'+num_str[4:-2]+'/'+num_str[0:4]
    
    return date_str 

VIX_TS['DATE'] = VIX_TS['DATE'].apply(DATE_STR)
VIX_TS['DATE'] = pd.to_datetime(VIX_TS['DATE'],format = '%d/%m/%Y')

### Load in VIX Future Returns

In [944]:
VIX_FUT = pd.read_csv(DATA_PATH + '/vixfut_ret.csv',skiprows=5)

VIX_FUT = VIX_FUT.rename({'Date':'DATE','r_1^{VIX fut.}':'1 month','r_2^{VIX fut.}':'2 month','r_3^{VIX fut.}':'3 month',
                         'r_4^{VIX fut.}':'4 month','r_5^{VIX fut.}':'5 month','r_6^{VIX fut.}':'6 month'},axis=1)

In [945]:
VIX_FUT['DATE'] = VIX_FUT['DATE'].apply(DATE_STR)
VIX_FUT['DATE'] = pd.to_datetime(VIX_FUT['DATE'],format = '%d/%m/%Y')

In [946]:
column_mapping = {
    '1 month': 'V_F_R 1 month',
    '2 month': 'V_F_R 2 month',
    '3 month': 'V_F_R 3 month',
    '4 month': 'V_F_R 6 month',
    '5 month': 'V_F_R 9 month',
    '6 month': 'V_F_R 12 month',
}

VIX_FUT.rename(columns=column_mapping, inplace=True)

### Load in Variance Swap Returns

In [947]:
VAR_SWAPS = pd.read_csv(DATA_PATH + '/vswap_ret.csv',skiprows=5)

VAR_SWAPS = VAR_SWAPS.rename({'Date':'DATE','r_1^{var. swap}':'1 month','r_2^{var. swap}':'2 month','r_3^{var. swap}':'3 month',
                         'r_6^{var. swap}':'6 month','r_9^{var. swap}':'9 month','r_12^{var. swap}':'12 month'},axis=1)

In [948]:
VAR_SWAPS['DATE'] = VAR_SWAPS['DATE'].apply(DATE_STR)
VAR_SWAPS['DATE'] = pd.to_datetime(VAR_SWAPS['DATE'],format = '%d/%m/%Y')

In [949]:
column_mapping = {
    '1 month': 'V_S_R 1 month',
    '2 month': 'V_S_R 2 month',
    '3 month': 'V_S_R 3 month',
    '6 month': 'V_S_R 6 month',
    '9 month': 'V_S_R 9 month',
    '12 month': 'V_S_R 12 month',
}

VAR_SWAPS.rename(columns=column_mapping, inplace=True)

### Load in Straddle Returns

In [950]:
STRADDLE = pd.read_csv(DATA_PATH + '/straddle_ret.csv',skiprows=6)

STRADDLE = STRADDLE.rename({'Date':'DATE',
                            'r_1^{straddle}':'1 month Long','r_2^{straddle}':'2 month Long','r_3^{straddle}':'3 month Long',
                            'r_6^{straddle}':'6 month Long','r_9^{straddle}':'9 month Long','r_12^{straddle}':'12 month Long',
                            'r_1^{short straddle}':'1 month Short','r_2^{short straddle}':'2 month Short','r_3^{short straddle}':'3 month Short',
                            'r_6^{short straddle}':'6 month Short','r_9^{short straddle}':'9 month Short','r_12^{short straddle}':'12 month Short',},axis=1)

In [951]:
STRADDLE['DATE'] = STRADDLE['DATE'].apply(DATE_STR)
STRADDLE['DATE'] = pd.to_datetime(STRADDLE['DATE'],format = '%d/%m/%Y')

In [952]:
STRADDLE_LONG = STRADDLE[['DATE','1 month Long','2 month Long','3 month Long','6 month Long','9 month Long','12 month Long']]

In [953]:
column_mapping = {
    '1 month Long': 'S_R 1 month',
    '2 month Long': 'S_R 2 month',
    '3 month Long': 'S_R 3 month',
    '6 month Long': 'S_R 6 month',
    '9 month Long': 'S_R 9 month',
    '12 month Long': 'S_R 12 month',
}

STRADDLE_LONG.rename(columns=column_mapping, inplace=True)

<ipython-input-953-1bc326bfab36>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STRADDLE_LONG.rename(columns=column_mapping, inplace=True)


### Load in Interest Rate Returns

In [954]:
df = pd.read_csv(DATA_PATH + '/IR.csv')

In [955]:
# Get rates for all days which are in the 300s
filtered_df = df[(df['days'] >= 300) & (df['days'] < 400)].reset_index(drop=True)

# Sort the DataFrame by 'dates' and 'days'
df_sorted = filtered_df.sort_values(by=['date', 'days'])

# Keep the rows with the closest 'days' value to 365 for each date
IR_TS = df_sorted.groupby('date', group_keys=False).apply(lambda group: group.iloc[(group['days'] - 365).abs().argsort()[:1]])

# Reset the index if needed
IR_TS.reset_index(drop=True, inplace=True)

In [956]:
IR_TS['Daily Rate'] = ((1+IR_TS['rate'])**(1/IR_TS['days']))-1
IR_TS = IR_TS.drop(['rate','days'],axis=1)

In [957]:
IR_TS.rename(columns={'date': 'DATE'}, inplace=True)

### Created Merged Databases for returns

In [958]:
IR_TS.set_index('DATE', inplace=True)
VIX_TS.set_index('DATE', inplace=True)
VAR_SWAPS.set_index('DATE', inplace=True)
VIX_FUT.set_index('DATE', inplace=True)
STRADDLE_LONG.set_index('DATE', inplace=True)

In [959]:
IR_TS.index = pd.to_datetime(IR_TS.index)
VIX_TS.index = pd.to_datetime(VIX_TS.index)
VAR_SWAPS.index = pd.to_datetime(VAR_SWAPS.index)
VIX_FUT.index = pd.to_datetime(VIX_FUT.index)
STRADDLE_LONG.index = pd.to_datetime(STRADDLE_LONG.index)

In [960]:
merged = pd.merge(VIX_TS,IR_TS, how='inner', left_index=True, right_index=True)

In [961]:
VAR_SWAPS = VAR_SWAPS/100
VIX_FUT = VIX_FUT/100
STRADDLE_LONG = STRADDLE_LONG/100

In [962]:
VAR_SWAPS_MERGED=pd.merge(merged,VAR_SWAPS, how='inner', left_index=True, right_index=True)
VIX_FUT_MERGED=pd.merge(merged,VIX_FUT, how='inner', left_index=True, right_index=True)
STRADDLE_MERGED=pd.merge(merged,STRADDLE_LONG, how='inner', left_index=True, right_index=True)

### Calculate Excess returns

In [963]:
XS_VAR_SWAPS = VAR_SWAPS_MERGED.iloc[:, 7:]
columns_to_subtract_from = ['V_S_R 1 month','V_S_R 2 month', 'V_S_R 3 month', 'V_S_R 6 month', 'V_S_R 9 month', 'V_S_R 12 month']

# Iterate through the columns and subtract 'column_to_subtract' from each of them
for col in columns_to_subtract_from:
    XS_VAR_SWAPS[col] = XS_VAR_SWAPS[col] - VAR_SWAPS_MERGED['Daily Rate']

In [964]:
XS_VIX_FUT = VIX_FUT_MERGED.iloc[:, 7:]
columns_to_subtract_from = ['V_F_R 1 month','V_F_R 2 month', 'V_F_R 3 month', 'V_F_R 6 month', 'V_F_R 9 month', 'V_F_R 12 month']

# Iterate through the columns and subtract 'column_to_subtract' from each of them
for col in columns_to_subtract_from:
    XS_VIX_FUT[col] = XS_VIX_FUT[col] - VIX_FUT_MERGED['Daily Rate']

In [965]:
XS_STRADDLE = STRADDLE_MERGED.iloc[:, 7:]
columns_to_subtract_from = ['S_R 1 month','S_R 2 month', 'S_R 3 month', 'S_R 6 month', 'S_R 9 month', 'S_R 12 month']

# Iterate through the columns and subtract 'column_to_subtract' from each of them
for col in columns_to_subtract_from:
    XS_STRADDLE[col] = XS_STRADDLE[col] - STRADDLE_MERGED['Daily Rate']

### Expectation Hypothesis Test

### Version 1

In [966]:
VIX_TS.reset_index(drop=True, inplace=True)

In [967]:
n_days = 21

In [968]:
Y_1 = VIX_TS['1 month'][n_days:].reset_index(drop=True) - VIX_TS['2 month'][:-n_days]
Y_2 = VIX_TS['2 month'][n_days:].reset_index(drop=True) - VIX_TS['3 month'][:-n_days]
Y_3 = VIX_TS['3 month'][3*n_days:].reset_index(drop=True) - VIX_TS['6 month'][:-3*n_days]
Y_4 = VIX_TS['6 month'][3*n_days:].reset_index(drop=True) - VIX_TS['9 month'][:-3*n_days]
Y_5 = VIX_TS['9 month'][3*n_days:].reset_index(drop=True) - VIX_TS['12 month'][:-3*n_days]

X_1 = (VIX_TS['2 month'][:-n_days] - VIX_TS['1 month'][:-n_days]).reset_index(drop=True)
X_2 = ((1/2)*VIX_TS['3 month'][:-n_days] - VIX_TS['1 month'][:-n_days]).reset_index(drop=True)
X_3 = (VIX_TS['6 month'][:-3*n_days] - VIX_TS['3 month'][:-3*n_days]).reset_index(drop=True)
X_4 = ((1/2)*VIX_TS['9 month'][:-3*n_days] - VIX_TS['3 month'][:-3*n_days]).reset_index(drop=True)
X_5 = ((1/3)*VIX_TS['12 month'][:-3*n_days] - VIX_TS['3 month'][:-3*n_days]).reset_index(drop=True)

In [969]:
data_1 = {'b_1': X_1,
        'b_2': X_2,
        'c_1': VIX_TS['1 month'][:-n_days].reset_index(drop=True),
       'c_2': VIX_TS['2 month'][:-n_days].reset_index(drop=True)}

data_2 = {'b_3': X_3,
        'b_4': X_4,
        'b_5': X_5,
       'c_3': VIX_TS['3 month'][:-3*n_days].reset_index(drop=True),
       'c_4': VIX_TS['6 month'][:-3*n_days].reset_index(drop=True),
       'c_5': VIX_TS['9 month'][:-3*n_days].reset_index(drop=True),}

df_1 = pd.DataFrame(data_1)
df_2 = pd.DataFrame(data_2)

In [970]:
X = df_1[['b_1','c_1']]
Y = Y_1

X = sm.add_constant(X)  # Add a constant (intercept) to the independent variables
model = sm.OLS(Y, X).fit()

# Calculate Newey West

cov_mat = stats.sandwich_covariance.cov_hac(model)
newey_west_se = np.sqrt(np.diag(cov_mat))

# Get the regression coefficients, including the intercept
coefficients = model.params

# Calculate p-values for the expectation hypothesis (b=1)
t_values = (coefficients[1]-1) / newey_west_se[1]

#Calculate p-values
p_values = 2 * scipy.stats.t.sf(abs(t_values), model.df_resid)

# Calculate R-squared
r_squared = model.rsquared

# Output the results
print("Regression coefficients (including intercept):", coefficients)
print("Newey-West standard errors:", newey_west_se)
print("P-values for the expectation hypothesis (b=1):", p_values.round(3))
print("R-squared:", r_squared)

Regression coefficients (including intercept): const    0.012449
b_1     -1.128694
c_1     -0.262035
dtype: float64
Newey-West standard errors: [0.00322047 0.29395276 0.0710115 ]
P-values for the expectation hypothesis (b=1): 0.0
R-squared: 0.09010903632954614


/Users/juliusraschke/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


### Version 2

In [971]:
Y_1 = VIX_TS['1 month'][n_days:].reset_index(drop=True) - VIX_TS['1 month'][:-n_days]
Y_2 = VIX_TS['2 month'][n_days:].reset_index(drop=True) - VIX_TS['2 month'][:-n_days]
Y_3 = VIX_TS['3 month'][3*n_days:].reset_index(drop=True) - VIX_TS['3 month'][:-3*n_days]
Y_4 = VIX_TS['6 month'][3*n_days:].reset_index(drop=True) - VIX_TS['6 month'][:-3*n_days]
Y_5 = VIX_TS['9 month'][3*n_days:].reset_index(drop=True) - VIX_TS['9 month'][:-3*n_days]

X_1 = (VIX_TS['2 month'][:-n_days] - VIX_TS['1 month'][:-n_days])+(VIX_TS['2 month'][:-n_days] - VIX_TS['1 month'][:-n_days])
X_2 = (VIX_TS['3 month'][:-n_days] - VIX_TS['2 month'][:-n_days])+((1/2)*VIX_TS['3 month'][:-n_days] - VIX_TS['1 month'][:-n_days])
X_3 = (VIX_TS['6 month'][:-3*n_days] - VIX_TS['3 month'][:-3*n_days])+(VIX_TS['6 month'][:-3*n_days] - VIX_TS['3 month'][:-3*n_days])
X_4 = (VIX_TS['9 month'][:-3*n_days] - VIX_TS['6 month'][:-3*n_days])+((1/2)*VIX_TS['9 month'][:-3*n_days] - VIX_TS['3 month'][:-3*n_days])
X_5 = (VIX_TS['12 month'][:-3*n_days] - VIX_TS['9 month'][:-3*n_days])+((1/3)*VIX_TS['12 month'][:-3*n_days] - VIX_TS['3 month'][:-3*n_days])

In [972]:
data_1 = {'b_1': X_1,
        'b_2': X_2,
        'c_1': VIX_TS['1 month'][:-n_days].reset_index(drop=True),
       'c_2': VIX_TS['2 month'][:-n_days].reset_index(drop=True)}

data_2 = {'b_3': X_3,
        'b_4': X_4,
        'b_5': X_5,
       'c_3': VIX_TS['3 month'][:-3*n_days].reset_index(drop=True),
       'c_4': VIX_TS['6 month'][:-3*n_days].reset_index(drop=True),
       'c_5': VIX_TS['9 month'][:-3*n_days].reset_index(drop=True),}

df_1 = pd.DataFrame(data_1)
df_2 = pd.DataFrame(data_2)

In [973]:
X = df_1[['b_1','c_1']]
Y = Y_1

X = sm.add_constant(X)  # Add a constant (intercept) to the independent variables

model = sm.OLS(Y, X).fit()

# Calculate Newey West

cov_mat = stats.sandwich_covariance.cov_hac(model)
newey_west_se = np.sqrt(np.diag(cov_mat))

# Get the regression coefficients, including the intercept
coefficients = model.params

# Calculate p-values for the expectation hypothesis (b=1)
t_values = (coefficients[1]-1) / newey_west_se[1]

#Calculate p-values
p_values = 2 * scipy.stats.t.sf(abs(t_values), model.df_resid)

# Calculate R-squared
r_squared = model.rsquared

# Output the results
print("Regression coefficients (including intercept):", coefficients)
print("Newey-West standard errors:", newey_west_se)
print("P-values for the expectation hypothesis (b=1):", p_values.round(3))
print("R-squared:", r_squared)

/Users/juliusraschke/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


Regression coefficients (including intercept): const    0.012449
b_1     -0.064347
c_1     -0.262035
dtype: float64
Newey-West standard errors: [0.00322047 0.14697638 0.0710115 ]
P-values for the expectation hypothesis (b=1): 0.0
R-squared: 0.12436506789993929


### Converting daily to monthly data

In [974]:
cr_VAR_SWAPS = ((XS_VAR_SWAPS)+1).rolling(n_days).apply(np.prod)-1
cr_VAR_SWAPS[:-(n_days-1):] = cr_VAR_SWAPS[(n_days-1)::]
cr_VAR_SWAPS = cr_VAR_SWAPS[:-(n_days-1):]

cr_VIX_FUT = ((XS_VIX_FUT)+1).rolling(n_days).apply(np.prod)-1
cr_VIX_FUT[:-(n_days-1):] = cr_VIX_FUT[(n_days-1)::]
cr_VIX_FUT = cr_VIX_FUT[:-(n_days-1):]

cr_STRADDLE = ((XS_STRADDLE)+1).rolling(n_days).apply(np.prod)-1
cr_STRADDLE[:-(n_days-1):] = cr_STRADDLE[(n_days-1)::]
cr_STRADDLE = cr_STRADDLE[:-(n_days-1):]

### Single Factor Test

In [975]:
pca1 = PCA()
scaler = StandardScaler()

### S&P 500 variance Swap Returns

In [976]:
X = VAR_SWAPS_MERGED.iloc[:,:6]
X_trans = pca1.fit_transform(X)
X_trans = pd.DataFrame(data=X_trans)
scaled = scaler.fit_transform(X_trans)
scaled_VS = pd.DataFrame(data=scaled)
scaled_VS[['Level','Slope','Curve','PC4','PC5','PC6']] = scaled_VS[[0,1,2,3,4,5]]
scaled_VS = scaled_VS.drop([0,1,2,3,4,5],axis=1)
scaled_VS['Slope'] = -scaled_VS['Slope']

### Next day

In [977]:
XS_V_S = XS_VAR_SWAPS
XS_V_S.reset_index(drop=False, inplace=True)

In [978]:
X = scaled_VS[:-1]
Y = (XS_V_S['V_S_R 9 month'][1:]*10000).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':0})

### Next month

In [979]:
cr_V_S = cr_VAR_SWAPS
cr_V_S.reset_index(drop=False, inplace=True)

In [980]:
X = scaled_VS[:-n_days]
Y = (cr_V_S['V_S_R 1 month'][1:]*100).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':n_days})

### VIX Futures Returns

In [981]:
X = VIX_FUT_MERGED.iloc[:,:6]
X_trans = pca1.fit_transform(X)
X_trans = pd.DataFrame(data=X_trans)
scaled = scaler.fit_transform(X_trans)
scaled_VF = pd.DataFrame(data=scaled)
scaled_VF[['Level','Slope','Curve','PC4','PC5','PC6']] = scaled_VF[[0,1,2,3,4,5]]
scaled_VF = scaled_VF.drop([0,1,2,3,4,5],axis=1)
scaled_VF['Slope'] = -scaled_VF['Slope']

### Next day

In [982]:
XS_V_F = XS_VIX_FUT
XS_V_F.reset_index(drop=False, inplace=True)

In [983]:
X = scaled_VF[:-1]
Y = (XS_V_F['V_F_R 9 month'][1:]*10000).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':0})

### Next month

In [984]:
cr_V_F = cr_VIX_FUT
cr_V_F.reset_index(drop=False, inplace=True)

In [985]:
X = scaled_VF[:-n_days]
Y = (cr_V_F['V_F_R 1 month'][1:]*100).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':n_days})

### Straddle Returns

In [986]:
X = STRADDLE_MERGED.iloc[:,:6]
X_trans = pca1.fit_transform(X)
X_trans = pd.DataFrame(data=X_trans)
scaled = scaler.fit_transform(X_trans)
scaled_S = pd.DataFrame(data=scaled)
scaled_S[['Level','Slope','Curve','PC4','PC5','PC6']] = scaled_S[[0,1,2,3,4,5]]
scaled_S = scaled_S.drop([0,1,2,3,4,5],axis=1)
scaled_S['Slope'] = -scaled_S['Slope']

### Next day

In [987]:
XS_S = XS_STRADDLE
XS_S.reset_index(drop=False, inplace=True)

In [988]:
X = scaled_S[:-1]
Y = (XS_S['S_R 9 month'][1:]*10000).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':0})

### Next month

In [989]:
cr_S = cr_STRADDLE
cr_S.reset_index(drop=False, inplace=True)

In [990]:
X = scaled_S[:-n_days]
Y = (cr_S['S_R 1 month'][1:]*100).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':n_days})

### Economic Significance of Slope as a predictor

In [991]:
ES_VS = XS_VAR_SWAPS[1:].reset_index(drop=True)
ES_VS['Slope_Quantile'] = pd.qcut(scaled_VS['Slope'], q=5, labels=False)[:-1]
VS_Q_Ret = ES_VS.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(VS_Q_Ret.min() - VS_Q_Ret.max()).T
range_row.index = ['Range']
VS_Q_Ret = pd.concat([VS_Q_Ret, range_row])
VS_Q_Ret

,V_S_R 1 month,V_S_R 2 month,V_S_R 3 month,V_S_R 6 month,V_S_R 9 month,V_S_R 12 month
0,-0.572038,0.244345,0.386122,0.258562,0.221150,0.182670
1,-1.399156,-0.728917,-0.424027,-0.191416,-0.120554,-0.085899
2,-1.521182,-0.828034,-0.585539,-0.306919,-0.206864,-0.105189
3,-1.490982,-0.903713,-0.533069,-0.376865,-0.181283,-0.023935
4,-2.005382,-1.346360,-0.987533,-0.614128,-0.502718,-0.495175
Range,-1.433343,-1.590706,-1.373655,-0.872690,-0.723868,-0.677846


In [992]:
ES_VF = XS_VIX_FUT[1:].reset_index(drop=True)
ES_VF['Slope_Quantile'] = pd.qcut(scaled_VF['Slope'], q=5, labels=False)[:-1]
VF_Q_Ret = ES_VF.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(VF_Q_Ret.min() - VF_Q_Ret.max()).T
range_row.index = ['Range']
VF_Q_Ret = pd.concat([VF_Q_Ret, range_row])
VF_Q_Ret

,V_F_R 1 month,V_F_R 2 month,V_F_R 3 month,V_F_R 6 month,V_F_R 9 month,V_F_R 12 month
0,0.056242,0.027001,0.000456,0.094582,0.108885,0.027323
1,-0.262411,-0.243983,-0.257679,-0.207549,-0.179378,-0.211195
2,-0.201813,-0.223828,-0.208229,-0.162340,-0.138415,-0.050902
3,-0.140691,-0.127868,-0.087659,-0.008243,-0.027570,-0.095389
4,-0.506191,-0.524580,-0.326711,-0.337252,-0.284948,-0.227393
Range,-0.562433,-0.551580,-0.327167,-0.431833,-0.393833,-0.254716


In [993]:
ES_STRADDLE =XS_STRADDLE[1:].reset_index(drop=True)
ES_STRADDLE['Slope_Quantile'] = pd.qcut(scaled_S['Slope'], q=5, labels=False)[:-1]
S_Q_Ret = ES_STRADDLE.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(S_Q_Ret.min() - S_Q_Ret.max()).T
range_row.index = ['Range']
S_Q_Ret = pd.concat([S_Q_Ret, range_row])
S_Q_Ret

,S_R 1 month,S_R 2 month,S_R 3 month,S_R 6 month,S_R 9 month,S_R 12 month
0,0.305044,0.296308,0.352745,0.302683,0.249758,0.212182
1,-0.371854,-0.220060,-0.161207,-0.101416,-0.052046,-0.024807
2,-0.420836,-0.268379,-0.176329,-0.104973,-0.071581,-0.047470
3,-0.472459,-0.311763,-0.225431,-0.101285,-0.075660,-0.048131
4,-0.653321,-0.433354,-0.342363,-0.205976,-0.155590,-0.094904
Range,-0.958364,-0.729663,-0.695108,-0.508659,-0.405348,-0.307086


### Removing Crisis

In [994]:
CRASH_VS_I = [VAR_SWAPS_MERGED.index.get_loc('2008-01-02'),VAR_SWAPS_MERGED.index.get_loc('2009-12-31')]
CRASH_VF_I = [VIX_FUT_MERGED.index.get_loc('2008-01-02'),VIX_FUT_MERGED.index.get_loc('2009-12-31')]
CRASH_S_I = [STRADDLE_MERGED.index.get_loc('2008-01-02'),STRADDLE_MERGED.index.get_loc('2009-12-31')]

In [995]:
scaled_NC_VS = pd.concat([scaled_VS[:CRASH_VS_I[0]],scaled_VS[CRASH_VS_I[1]:]]) 
scaled_NC_VF = pd.concat([scaled_VF[:CRASH_VF_I[0]],scaled_VF[CRASH_VF_I[1]:]]) 
scaled_NC_S = pd.concat([scaled_S[:CRASH_S_I[0]],scaled_S[CRASH_S_I[1]:]]) 

In [996]:
XS_NC_VS = pd.concat([XS_VAR_SWAPS[:CRASH_VS_I[0]],XS_VAR_SWAPS[CRASH_VS_I[1]:]]) 
XS_NC_VF = pd.concat([XS_VIX_FUT[:CRASH_VF_I[0]],XS_VIX_FUT[CRASH_VF_I[1]:]]) 
XS_NC_S = pd.concat([XS_STRADDLE[:CRASH_S_I[0]],XS_STRADDLE[CRASH_S_I[1]:]]) 

### Robustness of Slope as a predictor

### Variance Swap

### Next-month Returns

In [997]:
m_ES_VS = cr_VAR_SWAPS
m_ES_VS['Slope_Quantile'] = pd.qcut(scaled_VS['Slope'], q=5, labels=False)
m_VS_Q_Ret = m_ES_VS.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(m_VS_Q_Ret.min() - m_VS_Q_Ret.max()).T
range_row.index = ['Next-Month Returns']
m_VS_Q_Ret = pd.concat([m_VS_Q_Ret, range_row])

In [998]:
VS_Robustness = VS_Q_Ret.iloc[5:,:]
VS_Robustness = pd.concat([VS_Robustness, m_VS_Q_Ret.iloc[5:,:]])

### Crisis Removed

In [999]:
NC_ES_VS = XS_NC_VS[1:].reset_index(drop=True)
NC_ES_VS['Slope_Quantile'] = pd.qcut(scaled_NC_VS['Slope'], q=5, labels=False)
NC_VS_Q_Ret = NC_ES_VS.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(NC_VS_Q_Ret.min() - NC_VS_Q_Ret.max()).T
range_row.index = ['Crisis Removed']
NC_VS_Q_Ret = pd.concat([NC_VS_Q_Ret, range_row])

In [1000]:
VS_Robustness = pd.concat([VS_Robustness, NC_VS_Q_Ret.iloc[5:,:]])

### Bottom 5% Slope removed

In [1001]:
ES_VS = XS_VAR_SWAPS[1:].reset_index(drop=True)
ES_VS['Slope'] = scaled_VS['Slope']
ES_VS['Slope_Quantile'] = pd.qcut(scaled_VS['Slope'], q=5, labels=False)
ES_VS = ES_VS[ES_VS['Slope'] > ES_VS['Slope'].quantile(0.05)]
ES_VS = ES_VS.drop('Slope',axis=1)
BFIVE_VS_Q_Ret = ES_VS.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(BFIVE_VS_Q_Ret.min() - BFIVE_VS_Q_Ret.max()).T
range_row.index = ['Bottom 5% Slope removed']
BFIVE_VS_Q_Ret = pd.concat([BFIVE_VS_Q_Ret, range_row])

In [1002]:
VS_Robustness = pd.concat([VS_Robustness, BFIVE_VS_Q_Ret.iloc[5:,:]])

### Slope = VIX12 - VIX1

In [1003]:
NM_ES_VS = XS_VAR_SWAPS[1:].reset_index(drop=True)
NM_ES_VS['Slope_Quantile'] = pd.qcut((VAR_SWAPS_MERGED['12 month'] - VAR_SWAPS_MERGED['1 month']), q=5, labels=False).reset_index(drop=True)
NM_VS_Q_Ret = NM_ES_VS.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(NM_VS_Q_Ret.min() - NM_VS_Q_Ret.max()).T
range_row.index = ['Slope = VIX 12-VIX 1']
NM_VS_Q_Ret = pd.concat([NM_VS_Q_Ret, range_row])

In [1004]:
VS_Robustness = pd.concat([VS_Robustness, NM_VS_Q_Ret.iloc[5:,:]])

In [1005]:
VS_Robustness

,V_S_R 1 month,V_S_R 2 month,V_S_R 3 month,V_S_R 6 month,V_S_R 9 month,V_S_R 12 month
Range,-1.433343,-1.590706,-1.373655,-0.872690,-0.723868,-0.677846
Next-Month Returns,-35.525924,-33.029879,-27.574965,-17.765268,-14.299446,-10.653571
Crisis Removed,-1.213776,-1.304554,-1.093004,-0.791054,-0.710205,-0.657262
Bottom 5% Slope removed,-1.629203,-1.583190,-1.184340,-0.799831,-0.684337,-0.632386
Slope = VIX 12-VIX 1,-1.293989,-1.292387,-1.246605,-0.712265,-0.574801,-0.625708


### VIX Futures

### Next-month Returns

In [1006]:
m_ES_VF = cr_VIX_FUT
m_ES_VF['Slope_Quantile'] = pd.qcut(scaled_VF['Slope'], q=5, labels=False)
m_VF_Q_Ret = m_ES_VF.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(m_VF_Q_Ret.min() - m_VF_Q_Ret.max()).T
range_row.index = ['Next-Month Returns']
m_VF_Q_Ret = pd.concat([m_VF_Q_Ret, range_row])

In [1007]:
VF_Robustness = VF_Q_Ret.iloc[5:,:]
VF_Robustness = pd.concat([VF_Robustness, m_VF_Q_Ret.iloc[5:,:]])

### Crisis Removed

In [1008]:
NC_ES_VF = XS_NC_VF[1:].reset_index(drop=True)
NC_ES_VF['Slope_Quantile'] = pd.qcut(scaled_NC_VF['Slope'], q=5, labels=False)
NC_VF_Q_Ret = NC_ES_VF.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(NC_VF_Q_Ret.min() - NC_VF_Q_Ret.max()).T
range_row.index = ['Crisis Removed']
NC_VF_Q_Ret = pd.concat([NC_VF_Q_Ret, range_row])

In [1009]:
VF_Robustness = pd.concat([VF_Robustness, NC_VF_Q_Ret.iloc[5:,:]])

### Bottom 5% Slope removed

In [1010]:
ES_VF = XS_VIX_FUT[1:].reset_index(drop=True)
ES_VF['Slope'] = scaled_VF['Slope']
ES_VF['Slope_Quantile'] = pd.qcut(scaled_VF['Slope'], q=5, labels=False)
ES_VF = ES_VF[ES_VF['Slope'] > ES_VF['Slope'].quantile(0.05)]
ES_VF = ES_VF.drop('Slope',axis=1)
BFIVE_VF_Q_Ret = ES_VF.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(BFIVE_VF_Q_Ret.min() - BFIVE_VF_Q_Ret.max()).T
range_row.index = ['Bottom 5% Slope removed']
BFIVE_VF_Q_Ret = pd.concat([BFIVE_VF_Q_Ret, range_row])

In [1011]:
VF_Robustness = pd.concat([VF_Robustness, BFIVE_VF_Q_Ret.iloc[5:,:]])

In [1012]:
VF_Robustness

,V_F_R 1 month,V_F_R 2 month,V_F_R 3 month,V_F_R 6 month,V_F_R 9 month,V_F_R 12 month
Range,-0.562433,-0.551580,-0.327167,-0.431833,-0.393833,-0.254716
Next-Month Returns,-13.217869,-12.878687,-7.776461,-9.887845,-8.765469,-6.539853
Crisis Removed,-0.328988,-0.273191,-0.207673,-0.214958,-0.177985,-0.252202
Bottom 5% Slope removed,-0.405200,-0.396712,-0.239052,-0.329009,-0.257379,-0.176491


### Slope = VIX12 - VIX1

In [1013]:
NM_ES_VF = XS_VIX_FUT[1:].reset_index(drop=True)
NM_ES_VF['Slope_Quantile'] = pd.qcut((VIX_FUT_MERGED['12 month'] - VIX_FUT_MERGED['1 month']), q=5, labels=False).reset_index(drop=True)
NM_VF_Q_Ret = NM_ES_VF.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(NM_VF_Q_Ret.min() - NM_VF_Q_Ret.max()).T
range_row.index = ['Slope = VIX 12-VIX 1']
NM_VF_Q_Ret = pd.concat([NM_VF_Q_Ret, range_row])

In [1014]:
VF_Robustness = pd.concat([VF_Robustness, NM_VF_Q_Ret.iloc[5:,:]])

In [1015]:
VF_Robustness

,V_F_R 1 month,V_F_R 2 month,V_F_R 3 month,V_F_R 6 month,V_F_R 9 month,V_F_R 12 month
Range,-0.562433,-0.551580,-0.327167,-0.431833,-0.393833,-0.254716
Next-Month Returns,-13.217869,-12.878687,-7.776461,-9.887845,-8.765469,-6.539853
Crisis Removed,-0.328988,-0.273191,-0.207673,-0.214958,-0.177985,-0.252202
Bottom 5% Slope removed,-0.405200,-0.396712,-0.239052,-0.329009,-0.257379,-0.176491
Slope = VIX 12-VIX 1,-0.410109,-0.500155,-0.440284,-0.342576,-0.337151,-0.470074


### Straddle

### Next-month Returns

In [1016]:
m_ES_S = cr_STRADDLE
m_ES_S['Slope_Quantile'] = pd.qcut(scaled_S['Slope'], q=5, labels=False)
m_S_Q_Ret = m_ES_S.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(m_S_Q_Ret.min() - m_S_Q_Ret.max()).T
range_row.index = ['Next-Month Returns']
m_S_Q_Ret = pd.concat([m_S_Q_Ret, range_row])

In [1017]:
S_Robustness = S_Q_Ret.iloc[5:,:]
S_Robustness = pd.concat([S_Robustness, m_S_Q_Ret.iloc[5:,:]])

### Crisis Removed

In [1018]:
NC_ES_S = XS_NC_S[1:].reset_index(drop=True)
NC_ES_S['Slope_Quantile'] = pd.qcut(scaled_NC_S['Slope'], q=5, labels=False)
NC_S_Q_Ret = NC_ES_S.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(NC_S_Q_Ret.min() - NC_S_Q_Ret.max()).T
range_row.index = ['Crisis Removed']
NC_S_Q_Ret = pd.concat([NC_S_Q_Ret, range_row])

In [1019]:
S_Robustness = pd.concat([S_Robustness, NC_S_Q_Ret.iloc[5:,:]])

### Bottom 5% Slope removed

In [1020]:
ES_S = XS_STRADDLE[1:].reset_index(drop=True)
ES_S['Slope'] = scaled_S['Slope']
ES_S['Slope_Quantile'] = pd.qcut(scaled_S['Slope'], q=5, labels=False)
ES_S = ES_S[ES_S['Slope'] > ES_S['Slope'].quantile(0.05)]
ES_S = ES_S.drop('Slope',axis=1)
BFIVE_S_Q_Ret = ES_S.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(BFIVE_S_Q_Ret.min() - BFIVE_S_Q_Ret.max()).T
range_row.index = ['Bottom 5% Slope removed']
BFIVE_S_Q_Ret = pd.concat([BFIVE_S_Q_Ret, range_row])

In [1021]:
S_Robustness = pd.concat([S_Robustness, BFIVE_S_Q_Ret.iloc[5:,:]])

### Slope = VIX12 - VIX1

In [1022]:
NM_ES_S = XS_STRADDLE[1:].reset_index(drop=True)
NM_ES_S['Slope_Quantile'] = pd.qcut((STRADDLE_MERGED['12 month'] - STRADDLE_MERGED['1 month']), q=5, labels=False).reset_index(drop=True)
NM_S_Q_Ret = NM_ES_S.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(NM_S_Q_Ret.min() - NM_S_Q_Ret.max()).T
range_row.index = ['Slope = VIX 12-VIX 1']
NM_S_Q_Ret = pd.concat([NM_S_Q_Ret, range_row])

In [1023]:
S_Robustness = pd.concat([S_Robustness, NM_S_Q_Ret.iloc[5:,:]])

In [1024]:
S_Robustness

,S_R 1 month,S_R 2 month,S_R 3 month,S_R 6 month,S_R 9 month,S_R 12 month
Range,-0.958364,-0.729663,-0.695108,-0.508659,-0.405348,-0.307086
Next-Month Returns,-18.394685,-13.368522,-12.541455,-8.882468,-7.542562,-6.218885
Crisis Removed,-0.815367,-0.629175,-0.580671,-0.462567,-0.344703,-0.292992
Bottom 5% Slope removed,-0.945486,-0.657269,-0.592211,-0.423229,-0.334038,-0.257097
Slope = VIX 12-VIX 1,-0.697666,-0.539523,-0.498109,-0.320465,-0.246951,-0.221521


### Slope as an Incremental Predictor of Variance Asset Returns

In [1044]:
Y = XS_STRADDLE.iloc[:, 1:][1:].reset_index(drop=True)

In [1060]:
X1 = scaled_S['Slope'][:-1]
X2 = XS_STRADDLE.iloc[:, 1:][:-1]

X4 = STRADDLE_MERGED.iloc[:,:6][:-1].reset_index(drop=True)